<a href="https://colab.research.google.com/github/Run-aivle/study/blob/main/March/03.13/SEN/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random as rd
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras

In [2]:
(train_x, train_y), (test_x, test_y) = keras.datasets.fashion_mnist.load_data()

- MinMaxScaling

In [3]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [4]:
min_n, max_n = train_x.min(), train_x.max()
train_x = (train_x - min_n)/(max_n - min_n)
test_x = (test_x - min_n)/(max_n - min_n)

In [5]:
train_x.shape, test_x.shape

((60000, 28, 28), (10000, 28, 28))

- Reshape

In [6]:
train_x = train_x.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2], 1)
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2], 1)

print(train_x.shape, test_x.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


- to_categorical

In [7]:
from tensorflow.keras.utils import to_categorical
len_n = len(set(train_y))
train_y = to_categorical(train_y, len_n)
test_y = to_categorical(test_y, len_n)

In [8]:
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential

--------------------------------------------------------------
1. 인풋 레이어
2. Convolution : 필터수 32개, 사이즈(3, 3), same padding
3. BatchNormalization
4. Convolution : 필터수 32개, 사이즈(3, 3), same padding
5. BatchNormalization
6. MaxPooling : 사이즈(2,2) 스트라이드(2,2)
7. DropOut : 25% 비활성화
8. Convolution : 필터수 64개, 사이즈(3, 3), same padding
9. BatchNormalization
10. Convolution : 필터수 64개, 사이즈(3, 3), same padding
11. BatchNormalization
12. MaxPooling : 사이즈(2,2) 스트라이드(2,2)
13. DropOut : 25% 비활성화
14. Flatten
15. Fully Connected Layer : 노드 512개
16. BatchNormalization
17. 아웃풋 레이어
--------------------------------------------------------------

In [9]:
# Seqeuntial()

# 세션 클리어
clear_session()

# 모델 선언
model = Sequential()

# 모델 블록 조립
model.add(Input(shape=(28,28,1)))
model.add(Conv2D(filters = 32, 
                 kernel_size = (3,3),
                 strides = (1,1), 
                 padding = 'same', 
                 activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, 
                 kernel_size = (3,3),
                 strides = (1,1), 
                 padding = 'same', 
                 activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2,2),
                    strides = (2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 strides = (1,1), 
                 padding = 'same', 
                 activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 strides = (1,1), 
                 padding = 'same', 
                 activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2,2),
                    strides = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())

model.add(Dense(train_y.shape[1], activation = 'softmax'))


# 컴파일
model.compile(loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'],
              optimizer = 'adam')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                        

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor= 'val_loss',
                   min_delta = 0,
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

In [12]:
# 데이터를 넣어서 학습시키자!
model.fit(train_x, train_y, epochs=15, batch_size=1024,
             validation_split=0.2, callbacks = [es], verbose=1)

Epoch 1/15
47/47 [==============================] - 8s 37ms/step - loss: 0.5442 - accuracy: 0.8167 - val_loss: 2.4775 - val_accuracy: 0.0983
Epoch 2/15
47/47 [==============================] - 1s 24ms/step - loss: 0.2993 - accuracy: 0.8898 - val_loss: 2.7255 - val_accuracy: 0.1733
Epoch 3/15
47/47 [==============================] - 1s 24ms/step - loss: 0.2555 - accuracy: 0.9064 - val_loss: 2.6703 - val_accuracy: 0.1443
Epoch 4/15
47/47 [==============================] - 1s 24ms/step - loss: 0.2276 - accuracy: 0.9154 - val_loss: 2.9819 - val_accuracy: 0.1111
Epoch 5/15
47/47 [==============================] - 1s 24ms/step - loss: 0.2069 - accuracy: 0.9241 - val_loss: 3.1418 - val_accuracy: 0.1042
Epoch 6/15
47/47 [==============================] - 1s 24ms/step - loss: 0.1886 - accuracy: 0.9298 - val_loss: 3.4335 - val_accuracy: 0.1033
Epoch 6: early stopping


In [13]:
model.evaluate(train_x, train_y)
model.evaluate(test_x, test_y)

313/313 [==============================] - 1s 3ms/step - loss: 2.4793 - accuracy: 0.1000


[2.4793450832366943, 0.10000000149011612]